In [2]:
import pandas as pd
from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

diabetes = pd.read_csv('data_14/diabetes_data.csv')
diabetes.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Gender
0,6,98,58,33,190,34.0,0.430,43,0,Female
1,2,112,75,32,0,35.7,0.148,21,0,Female
2,2,108,64,0,0,30.8,0.158,21,0,Female
3,8,107,80,0,0,24.6,0.856,34,0,Female
4,7,136,90,0,0,29.9,0.210,50,0,Female


1. Начнём с поиска дубликатов в данных. Найдите все повторяющиеся строки в данных и удалите их. Для поиска используйте все признаки в данных. Сколько записей осталось в данных?

In [6]:
duplicates = diabetes[diabetes.duplicated()]
print(duplicates.shape[0], 'дублей')
diabetes = diabetes.drop_duplicates()
print('Число строк', diabetes.shape[0])

0 дублей
Число строк 768


2. Далее найдите все неинформативные признаки в данных и избавьтесь от них. В качестве порога информативности возьмите 0.99: удалите все признаки, для которых 99 % значений повторяются или 99 % записей уникальны. В ответ запишите имена признаков, которые вы нашли (без кавычек).

In [7]:
low_information_cols = [] #список неинформативных признаков

for col in diabetes.columns: #цикл по всем столбцам
    top_freq = diabetes[col].value_counts(normalize=True).max() #наибольшая относительная частота в признаке
    nunique_ratio = diabetes[col].nunique() / diabetes[col].count() #доля уникальных значений от размера признака
    if top_freq > 0.99: # сравниваем наибольшую частоту с порогом
        low_information_cols.append(col)
        print(f'{col}: {round(top_freq*100, 2)}% одинаковых значений')
    if nunique_ratio > 0.99: # сравниваем долю уникальных значений с порогом
        low_information_cols.append(col)
        print(f'{col}: {round(nunique_ratio*100, 2)}% уникальных значений')

Gender: 100.0% одинаковых значений


Попробуйте найти пропуски в данных с помощью метода isnull().
Спойлер: ничего не найдёте. А они есть! Просто они скрыты от наших глаз. В таблице пропуски в столбцах Glucose, BloodPressure, SkinThickness, Insulin и BMI обозначены нулём, поэтому традиционные методы поиска пропусков ничего вам не покажут. Давайте это исправим!
Замените все записи, равные 0, в столбцах Glucose, BloodPressure, SkinThickness, Insulin и BMI на символ пропуска. Его вы можете взять из библиотеки numpy: np.nan.
Какая доля пропусков содержится в столбце Insulin? Ответ округлите до сотых.

In [10]:
nul_cols_persent = diabetes.isnull().mean()*100
nul_cols = nul_cols_persent[nul_cols_persent > 0]
print(nul_cols)

Series([], dtype: float64)
